In [1]:
import itertools
import numpy as np
import time
import gpflow
import tensorflow as tf
import matplotlib.pyplot as plt
from gpflow.ci_utils import *
from gpflow.utilities import print_summary

In [2]:
tf.random.set_seed(123)

In [3]:
def run_adam(model, iterations):
    """
    Utility function running the Adam optimizer

    :param model: GPflow model
    :param interations: number of iterations
    """
    # Create an Adam Optimizer action
    logf = []
    training_loss = model.training_loss_closure(tensor_data, compile=True)
    optimizer = tf.optimizers.Adam()

    @tf.function
    def optimization_step():
        optimizer.minimize(training_loss, model.trainable_variables)

    for step in range(iterations):
        optimization_step()
        if step % 10 == 0:
            elbo = -training_loss().numpy()
            logf.append(elbo)
    return logf

In [4]:
d = 10
class Matern25_aniso(gpflow.kernels.AnisotropicStationary):
     def K_d(self, d):
            sqrt5 = np.sqrt(5.0)
            d = tf.square(d)
            d = tf.reduce_sum(d, -1)
            d = tf.sqrt(d)
            return self.variance * (1.0 + sqrt5 * d + 5.0 / 3.0 * tf.square(d)) * tf.exp(-sqrt5 * d)
Matern25_aniso(variance = 1.0, lengthscales = np.concatenate([np.repeat(20, 2), np.repeat(0.2, d - 2)]))

2021-10-15 12:56:30.737163: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


name,class,transform,prior,trainable,shape,dtype,value
Matern25_aniso.variance,Parameter,Softplus,,True,(),float64,1.0
Matern25_aniso.lengthscales,Parameter,Softplus,,True,"(10,)",float64,"[20., 20., 0.2..."


In [5]:
mMax = 52
for i in range(1, 11):
    with open("locs_" + str(i) + ".csv") as locsFile:
        locs = np.genfromtxt(locsFile, delimiter = ',').reshape((-1, d))
        locsInduce = locs[ : mMax, :]
        locs = locs[mMax :, :]
        y = np.zeros((locs.shape[0], 1))
        for M in range(4, 53, 4): # Number of inducing locations  
            data = (locs, y)
            kernel = Matern25_aniso(variance = 1.0, 
                                    lengthscales = np.concatenate([np.repeat(20, 2), np.repeat(0.2, d - 2)]))
            Z = locsInduce[: M, :].copy()  # Initialize inducing locations to the first M inputs in the dataset
            m = gpflow.models.SVGP(kernel, gpflow.likelihoods.Gaussian(), Z, num_data = y.size, mean_function=None)
            gpflow.set_trainable(m.kernel, False)
            gpflow.set_trainable(m.inducing_variable, True)
            tensor_data = tuple(map(tf.convert_to_tensor, data))
            maxiter = ci_niter(1000)
            run_adam(m, maxiter)
            np.savetxt("induce_" + str(i) + "_" + str(M) + ".csv",
                       np.array(m.inducing_variable.parameters[0]), delimiter=",")

2021-10-15 12:56:30.844352: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2021-10-15 12:56:34.507252: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
